In [1]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [2]:
# Your code here to find the table name
tables = client.list_tables(dataset)
for table in tables:  
    print(table.table_id)

taxi_trips


In [3]:
table_ref = dataset_ref.table('taxi_trips')
table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,8f7085c0a9632f3c462c5543cb9df420df1f4029,c51889ed8a7b4e546482f2bbe16bee57680f70542e78a7...,2016-10-23 16:45:00+00:00,2016-10-23 17:15:00+00:00,1601,13.6,<NA>,<NA>,<NA>,<NA>,...,0.0,0.01,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
1,9414ae6ac589eb8c92065bd8926aaa332fb62283,c51889ed8a7b4e546482f2bbe16bee57680f70542e78a7...,2016-10-23 17:30:00+00:00,2016-10-23 17:45:00+00:00,783,3.8,<NA>,<NA>,<NA>,<NA>,...,0.0,14.95,Credit Card,303 Taxi,NaN,NaN,None,NaN,NaN,None
2,a09cc3a425b81b87b990721a767da373364496a0,25f108dbdee2f40abf533ec54828d51019935abcf13c9a...,2018-01-14 15:15:00+00:00,2018-01-14 15:30:00+00:00,593,1.1,<NA>,<NA>,<NA>,<NA>,...,0.0,6.80,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
3,ce18b8ad09a687248fdf69f35e31dc6bd926effd,28cc6d01750f2d536f164256e6a563ac6e8031e88e8519...,2018-01-04 07:30:00+00:00,2018-01-04 07:30:00+00:00,305,1.2,<NA>,<NA>,<NA>,<NA>,...,0.0,5.20,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
4,37c4d9d1f0298df2d8151d2ad6f9a5c0306d10ff,28cc6d01750f2d536f164256e6a563ac6e8031e88e8519...,2018-01-04 08:00:00+00:00,2018-01-04 08:00:00+00:00,4,0.0,<NA>,<NA>,<NA>,<NA>,...,0.0,2.00,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None


In [4]:
# Rides per year
rides_per_year_query = """
                        WITH time AS
                        (
                             SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year
                             FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`                       
                        )
                        SELECT COUNT(1) as num_trips, year
                        FROM time
                        GROUP BY year
                        ORDER BY year
                        """

# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_year_query_job = client.query(rides_per_year_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
rides_per_year_result = rides_per_year_query_job.to_dataframe() # Your code goes here

# View results
print(rides_per_year_result)

   num_trips  year
0   27217716  2013
1   37395436  2014
2   32385875  2015
3   31759339  2016
4   24988003  2017
5   20732088  2018
6   16477365  2019
7    3889032  2020
8    3948045  2021
9    1713146  2022


In [5]:
# Rides per month
rides_per_month_query = """
                        WITH time AS
                        (
                             SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year, EXTRACT(MONTH FROM trip_start_timestamp) AS month
                             FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`                       
                        )
                        SELECT month, COUNT(1) as num_trips
                        FROM time
                        WHERE year = 2017
                        GROUP BY month
                        ORDER BY month
                        """ 

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_month_query_job = client.query(rides_per_month_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
rides_per_month_result = rides_per_month_query_job.to_dataframe() # Your code goes here

# View results
print(rides_per_month_result)

    month  num_trips
0       1    1972071
1       2    1909802
2       3    2362105
3       4    2194702
4       5    2323386
5       6    2324472
6       7    2054299
7       8    2079861
8       9    1950631
9      10    2141197
10     11    1907997
11     12    1767480


In [6]:
# Rides per hour
speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day, 
                          trip_miles, 
                          trip_seconds
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_start_timestamp > '2017-01-01' AND 
                         trip_start_timestamp < '2017-07-01' AND 
                         trip_seconds > 0 AND 
                         trip_miles > 0
               )
               SELECT hour_of_day, 
                      COUNT(1) AS num_trips, 
                      3600 * SUM(trip_miles) / SUM(trip_seconds) AS avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
speeds_query_job = client.query(speeds_query,job_config=safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
speeds_result = speeds_query_job.to_dataframe() # Your code here

# View results
print(speeds_result)

    hour_of_day  num_trips    avg_mph
0             0     319339  20.230524
1             1     266529  18.937621
2             2     210147  18.777070
3             3     159668  20.158048
4             4     122183  26.736014
5             5     119312  30.769172
6             6     182738  24.588313
7             7     358406  17.735967
8             8     541775  15.079892
9             9     565548  16.543882
10           10     525120  18.539614
11           11     594603  18.928379
12           12     622324  17.838745
13           13     630181  17.671089
14           14     622465  16.974239
15           15     640430  15.688418
16           16     701435  14.283888
17           17     756627  12.462955
18           18     768251  13.646810
19           19     701064  16.642882
20           20     598614  19.536777
21           21     552726  20.433874
22           22     501095  19.531374
23           23     399587  19.877046
